In [36]:
## Lets import our modules
import boto3
import pandas as pd
#import psycopg2
import pymysql
from sqlalchemy import create_engine
import os
import sqlite3
import sys
parentDirectory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0,parentDirectory)
from secretsPaul import secrets
import mysql.connector
import plotly.express as px
from collections import Counter
import nlp

In [32]:
#connections
#def create_connection():
endpoint = 'drown-murphy-lankheet.cluster-ceswqg9qwa9i.us-east-1.rds.amazonaws.com'#secrets.get('DATABASE_ENDPOINT')
user = secrets.get('DATABASE_USER')
password = secrets.get('DATABASE_PASSWORD')
port=secrets.get('DATABASE_PORT')
connection = create_engine(f'mysql+pymysql://{user}:{password}@{endpoint}:{port}/Capstone', pool_recycle=3600,  connect_args={'connect_timeout': 10});
    
connect=mysql.connector.connect(host=endpoint, database='Capstone', user=user, password=password)
cursor = connect.cursor()
    #return cursor
#cursor=create_connection()

In [33]:
#Pulling in listings
all_listings = pd.read_sql('Select scrape_city, amenities, review_scores_value  from recent_listings_occup',connection) 

In [34]:
#Pulling in reviews & hostnames for removal
all_reviews = pd.read_sql('SELECT comments, scrape_city FROM reviews',connection) 
host_names=pd.read_sql('Select Distinct host_name from listings',connection)
hostnames = set(host_names.host_name)
hostnames = {hn.lower() for hn in hostnames}

In [35]:
#unique list of all cities
scrape_list_=list(all_listings['scrape_city'].unique())

In [38]:
#Looping through each city and running the amenities
top_amenities=pd.DataFrame(columns=['amenity','score','scrape_city'])
top_reviews=pd.DataFrame(columns=['pos_word','pos_score','neg_word','neg_score','scrape_city'])
for city in scrape_list:
    topa=nlp.get_top_amenities(all_listings[all_listings['scrape_city']==city])
    topa['scrape_city']=city
    topa.reset_index(inplace=True)
    top_amenities=pd.concat([top_amenities, topa],ignore_index=True)
    topr=nlp.get_top_review_terms(all_reviews[all_reviews['scrape_city']==city],host_names)
    topr['scrape_city']=city
    top_reviews=pd.concat([top_reviews, topr],ignore_index=True)
    
    

C:\Users\paull\AppData\Local\Temp\ipykernel_33292\1784826504.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topr['scrape_city']=city


In [39]:
top_amenities.to_sql('top_amenities',con=connection, if_exists='replace',index=False, chunksize=1000, method='multi')
top_reviews.to_sql('top_reviews',con=connection, if_exists='replace',index=False, chunksize=1000, method='multi')

310